In [1]:
from src.project import Project

proj = Project('/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh','10_09_01MJPG-0000')

In [2]:
bg, calibration, feat, params, track = proj.read_project_data()

In [3]:
import pandas as pd
import numpy as np

def convert_data_to_df(data, col_names):
    n_flies, n_frames, n_flags = data.shape
    reshaped_data = data.reshape(-1, n_flags)

    df = pd.DataFrame(reshaped_data, columns=col_names)
    df['id'] = np.repeat(np.arange(n_flies), n_frames)

    return df

In [4]:
track_data = track['trk']['data']
feature_data = feat['feat']['data']

track_names = [i.replace(' ', '_') for i in track['trk']['names']]
feature_names = feat['feat']['names']

In [5]:
track_df = convert_data_to_df(track_data, track_names)
feature_df = convert_data_to_df(feature_data, feature_names)

In [6]:
# Concatenate along axis=1 (horizontally)
track_feature = pd.concat([feature_df, track_df], axis=1)

In [7]:
track_feature

,vel,ang_vel,min_wing_ang,max_wing_ang,mean_wing_length,axis_ratio,fg_body_ratio,contrast,dist_to_wall,dist_to_other,...,leg_5_y,leg_6_x,leg_6_y,leg_1_ang,leg_2_ang,leg_3_ang,leg_4_ang,leg_5_ang,leg_6_ang,id
0,27.760335,0.059031,0.070184,0.113363,1.904930,3.241620,3.105856,0.265803,2.909003,25.870275,...,NaN,NaN,NaN,-0.591355,1.570320,2.736041,NaN,NaN,NaN,0
1,27.760335,0.287484,0.060691,0.102101,1.902980,3.322472,3.112842,0.269980,2.683556,25.764119,...,128.0,NaN,NaN,-1.891430,-0.448297,0.890940,1.242397,2.760944,NaN,0
2,22.893499,0.987162,0.072418,0.099556,1.895876,3.391456,3.173063,0.268889,2.517328,25.661251,...,129.0,NaN,NaN,-2.927364,-1.520840,0.720157,1.561423,2.641311,NaN,0
3,15.135936,1.529196,0.080854,0.093614,1.895622,3.361646,3.224714,0.268279,2.412099,25.550027,...,128.0,NaN,NaN,-2.906173,-1.324485,0.654087,1.912728,2.493395,NaN,0
4,6.958983,1.393339,0.072333,0.095410,1.903336,3.249895,3.123369,0.276265,2.389758,25.459197,...,124.0,497.0,128.0,-2.955907,-1.266117,-0.723773,0.615680,1.790462,2.375198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215995,16.330244,0.594309,0.020885,0.063930,2.200976,3.192546,2.480808,0.166200,5.350743,3.954129,...,946.0,NaN,NaN,-1.336897,-0.981502,0.688471,1.778683,2.762487,NaN,1
215996,16.729621,0.968933,0.020889,0.051766,2.192903,3.237779,2.493921,0.167686,5.325563,3.998637,...,NaN,NaN,NaN,-1.901137,0.804953,1.642987,2.757793,NaN,NaN,1
215997,18.949663,1.013950,0.020647,0.048799,2.201918,3.258233,2.505767,0.167309,5.305263,4.060245,...,NaN,NaN,NaN,-1.916883,1.062256,1.480833,2.702737,NaN,NaN,1
215998,19.587339,0.867803,0.022211,0.056043,2.214864,3.249690,2.528554,0.165846,5.254227,4.092901,...,940.0,351.0,955.0,-3.005526,-1.666470,-0.691837,0.849302,1.813714,2.641627,1


In [ ]:
from src.project_dlc import ProjectDLC
from src.video import VideoInfo

complete_data_path = "/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh/dlc_output/20240115-1015-fly1-melWT_3do_sh_melWT_3do_ghDLC_dlcrnetms5_38mm-dyadFeb23shuffle1_50000_el.h5"

video_path = "/Users/isabellechen/Dropbox @RU Dropbox/Isabelle Chen/example_data/20240115-1015-fly1-melWT_3do_sh_melWT_3do_gh/10_09_01MJPG-0000.avi"
video = VideoInfo(video_path)
full_data = ProjectDLC.load_data(complete_data_path)

/Users/isabellechen/flytracker_analysis/src/dlc/measurement.py:32: RuntimeWarning: Mean of empty slice
  x_center, y_center = np.nanmean(x_coords, axis=1), np.nanmean(y_coords, axis=1)


In [ ]:
df_fly1 = ProjectDLC.process_single_fly(full_data, "fly1", video)
df_fly2 = ProjectDLC.process_single_fly(full_data, "fly2", video)

/Users/isabellechen/flytracker_analysis/src/dlc/measurement.py:32: RuntimeWarning: Mean of empty slice
  x_center, y_center = np.nanmean(x_coords, axis=1), np.nanmean(y_coords, axis=1)


In [10]:
df = ProjectDLC.merge_dfs_and_add_interfly_data(df_fly1, df_fly2, "Dmel")

In [11]:
df

,ori,centroid_x,centroid_y,lin_speed,left_wing_angle,right_wing_angle,inter_wing_dist,body_length,time,id,inter_obj_dist,fly1_facing_angle,fly1_ang_between,fly2_facing_angle,fly2_ang_between,species
0,1.092144,246.3790,815.3608,0.000000,-0.020222,0.005581,2.218317,65.573367,0.000000,0,714.948660,-2.211888,-2.580888,NaN,NaN,Dmel
1,0.851593,245.9306,814.8866,7.831581,-0.136659,-0.101541,2.331655,59.682535,0.016667,0,710.386278,-1.990429,-2.673250,NaN,NaN,Dmel
2,0.788169,247.0506,813.4030,22.306670,-0.123074,-0.083358,2.560145,62.533228,0.033334,0,707.847641,-1.944075,-2.746615,NaN,NaN,Dmel
3,0.802668,251.2940,813.9566,51.352312,-0.038331,0.075875,7.550308,73.091092,0.050000,0,707.906575,-1.971763,-2.707187,NaN,NaN,Dmel
4,0.837670,251.8430,814.6830,10.926314,0.047060,0.109841,4.336923,70.022846,0.066667,0,706.313020,-2.008824,-2.754267,NaN,NaN,Dmel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,0.674662,235.4796,793.3524,47.567674,-1.048371,0.122882,69.803410,61.592902,1799.933333,1,NaN,NaN,NaN,0.123692,-0.250883,Dmel
107996,0.735481,242.5540,800.8660,123.839372,-0.947431,0.212519,71.150434,58.329769,1799.950000,1,NaN,NaN,NaN,-0.045260,-0.138462,Dmel
107997,0.683635,233.5306,811.8246,170.346187,0.047687,0.151496,7.995680,54.661417,1799.966666,1,NaN,NaN,NaN,-0.260548,-0.193211,Dmel
107998,0.638478,253.2654,809.9130,237.926002,-1.055485,0.021859,71.060859,55.900105,1799.983333,1,NaN,NaN,NaN,0.226222,-0.230418,Dmel
